In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("Tensorflow-version:", tensorflow.__version__)

In [ ]:
model_d = DenseNet121(weights='imagenet', include_top = False, input_shape = (128,128, 3))
x = model_d.output

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512,activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

preds=Dense(2,activation='softmax')(x)

In [ ]:

model=Model(inputs=model_d.input,outputs=preds)
# model.summary()

In [ ]:
for layer in model.layers[:-8]:
  layer.trainable = False

for layer in model.layers[-8:]:
  layer.trainable = True 

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics = ['accuracy'])
# model.summary()

In [ ]:
data=[]
labels=[]
random.seed(42)

image_names = list(os.listdir("/content/gdrive/MyDrive/project-art/datasets/temp/train_2"))
impressionism_names = random.sample(list(filter(lambda x: 'impressionism' in x, image_names)), 60) # 60 stycken random samples av impressionism
surrealism_names = random.sample(list(filter(lambda x: 'surrealism' in x, image_names)), 60)
sample_image_names = impressionism_names + surrealism_names
random.shuffle(sample_image_names)

In [ ]:
data=[]
labels=[]

for name in sample_image_names:
  image = cv2.imread(f'/content/gdrive/MyDrive/project-art/datasets/temp/train_2/{name}')
  image = cv2.resize(image, (128, 128))
  image = img_to_array(image)
  data.append(image)
  if 'impressionism' in name:
    label = 'impressionism'
  else:
    label = 'surrealism'
  labels.append(label)

In [ ]:

print(len(labels))
print(len(data))

In [ ]:
from keras.utils import to_categorical
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(data,labels,test_size=0.4,random_state=42)
print(xtrain.shape, ytrain.shape)

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(xtrain)
print(xtrain.shape)

history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=72),
               steps_per_epoch=xtrain.shape[0] / 72,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))


In [ ]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
  if np.argmax(ypred[i]) == np.argmax(ytest[i]):
    accurate +=1
    accurateindex.append(i)
  else:
    wrongindex.append(i)

  total +=1

print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\twrongly-predicted-data:', total - accurate)
print('Accuracy', round(accurate/total*100, 3), '%')

In [ ]:
label=['impressionism', 'surrealism']
imidx = random.sample(accurateindex, k=9) # replace with 'wrongindex' accurateindex

nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex = True, sharey = True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(xtest[imidx[n]])
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(ytest[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
label=['impressionism', 'surrealism']
imidx = random.sample(wrongindex, k=4) # replace with 'wrongindex' accurateindex

nrows = 2
ncols = 2
fig, ax = plt.subplots(nrows,ncols,sharex = True, sharey = True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(xtest[imidx[n]])
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(ytest[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)
